## 6. Prediction - Random Forest Model!

### About this Method and the assumption we have behind it:
- There are Numbers which have been drawn slightly more than others
- We try our Luck and pick the 6 most frequently Drawn Numbers
- We do the same for the Superzahl

For our 1 Prediction (remember we play 6 Fields per Lottery tickey á 6 Numbers + 1 Superzahl (for the whole Ticket Superzahl can only be selected once))


In [7]:
# Import Libraries needed
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

### Load the Data we Stored in the data Folder with Notebook 1

In [8]:

# Load the CSV file into a DataFrame
frequency_data = pd.read_csv("data/frequency_data.csv")


In [9]:
# convert the string to datetime
frequency_data["date"] = pd.to_datetime(frequency_data["date"], format='%d.%m.%Y')

# Convert the Lottozahl column to a list of integers
frequency_data["Lottozahl"] = frequency_data["Lottozahl"].apply(lambda x: eval(x))

# Expand the Lottozahl column into seperate columns
df_expanded = pd.DataFrame(frequency_data["Lottozahl"].tolist(), columns=['Lottozahl_1', 'Lottozahl_2', 'Lottozahl_3', 'Lottozahl_4', 'Lottozahl_5', 'Lottozahl_6'])

# Create the final Dataframe
df_final = pd.concat([frequency_data.drop(columns= "Lottozahl"), df_expanded], axis=1)


# Cast the Date into own columns
df_final["day"] = df_final["date"].dt.day
df_final["month"] = df_final["date"].dt.month
df_final["year"] = df_final["date"].dt.year

# Drop the Superzahl Column (we only Predict the 6 Lotto Numbers)
df_final.drop(["Superzahl", "id","date"], axis=1, inplace=True)

df_final.head()

,Lottozahl_1,Lottozahl_2,Lottozahl_3,Lottozahl_4,Lottozahl_5,Lottozahl_6,day,month,year
0,3,12,13,16,23,41,9,10,1955
1,3,12,18,30,32,49,16,10,1955
2,12,14,23,24,34,36,23,10,1955
3,4,13,23,30,36,44,30,10,1955
4,5,6,31,39,44,49,6,11,1955


### Now we count the Frequency of the Numbers which were drawn since the beginning of the Records

In [10]:
# Assuming df is your DataFrame containing the data
X = df_final[['day', 'month', 'year']]  # Features: day, month, year
y = df_final[['Lottozahl_1', 'Lottozahl_2', 'Lottozahl_3', 'Lottozahl_4', 'Lottozahl_5', 'Lottozahl_6']]  # Target variables: Lotto numbers

In [11]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [12]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}


In [18]:
# define the model
rf = RandomForestRegressor(random_state = 42)

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_squared_error')

In [19]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Negative Mean Squared Error: ", best_score)

# Get the best model
best_rf_model = grid_search.best_estimator_

# Evaluate the best model
y_pred_best = best_rf_model.predict(X_test)
mse_best = mean_squared_error(y_test,y_pred_best)
print("Mean Squared Error for Best Model:", mse_best)



Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 150}
Best Negative Mean Squared Error:  -51.842453116582234
Mean Squared Error for Best Model: 54.34058743523866


## Adjust the Date here for the Date for which u want to have Numbers!

In [29]:

# Change the Day of the Drawing here
new_dates = np.array([[27, 4, 2024]])
# Create a DataFrame
new_dates = pd.DataFrame(new_dates, columns=['day', 'month', 'year'])

In [31]:
# Predict lotto numbers for new dates using the best model
new_predictions_best = best_rf_model.predict(new_dates)
print("Predicted Lotto Numbers for New Dates using Best Random Forrest Model:", new_predictions_best)

Predicted Lotto Numbers for New Dates using Best Random Forrest Model: [[ 5.34011002 12.58815013 19.66754466 28.08359562 36.06410518 43.01665058]]


# Predictions for the 1 Lottery Ticket Field!

### DISCLAIMER:
- This is a Fun Project to Showcase some IT Skills
- Lottery is a total statistically Random Game
- Do not use or let you inspire by this Prediction!
- Gambling can make you addicted! Only Play with Caution!
- I claim no rights, that my outputs would even be near the drawn value

### About the Predictions:
- In Field 1 we go for the Most Frequent Numbers each Lottery Ticket we play
- The Superzahl only appears once per Lottery Ticket - we will Take 7 and 4 randomly for each Ticket
- This is a common Lottery Strategy - even if you win with this Numbers, you would have to share the Prize Pool with many other People
- We will do more complex Prediction Strategies in later Notebooks
- This prediction will be the same for several Weeks till another number gets into Top 6